# Lesson 1-2: What Is an Agent? + Function Tools

In this notebook, we'll cover the foundational concepts of the OpenAI Agents SDK:

1. **What is an agent?** - An LLM equipped with instructions and tools
2. **The agent loop** - Tool call → execute → feed back → repeat
3. **Function tools** - `@function_tool` decorator with automatic schema generation
4. **Pydantic validation** - Type-safe tool parameters

## The Four Primitives

The OpenAI Agents SDK has exactly four primitives:
- **Agents**: LLMs equipped with instructions and tools
- **Tools**: Functions the agent can call
- **Handoffs**: Allow agents to delegate to other agents
- **Guardrails**: Validate agent inputs and outputs

Plus supporting infrastructure: Runner, Context, Sessions, Tracing.

## Setup

In [ ]:
# Install the SDK (uncomment if needed)
# !pip install openai-agents

In [ ]:
# Required for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Set your OpenAI API key
import os
import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

## Your First Agent: Hello World

The simplest possible agent - just an LLM with instructions. No tools, no complexity.

In [ ]:
from agents import Agent, Runner

# Create a simple agent
agent = Agent(
    name="Greeter",
    instructions="You are a friendly assistant. Keep responses brief.",
    model="gpt-4.1"
)

# Run the agent synchronously
result = Runner.run_sync(agent, "Hello! What can you do?")
print(result.final_output)

**That's it!** Five lines of code to create and run an agent:
1. Import `Agent` and `Runner`
2. Create an `Agent` with name, instructions, and model
3. Use `Runner.run_sync()` to execute

But without tools, this is just a chatbot. Let's make it useful.

## The Agent Loop

Here's the key concept: **an agent runs in a loop**.

```
User Input
    ↓
┌─────────────────────────────┐
│  LLM decides what to do    │
│  (respond or call a tool)  │
└─────────────────────────────┘
    ↓                    ↓
[Final Response]    [Tool Call]
    ↓                    ↓
  Done              Execute Tool
                         ↓
                   Feed result back
                         ↓
                   ┌─────────────┐
                   │ Loop again  │
                   └─────────────┘
```

The loop continues until the LLM produces a final response (no more tool calls).

**This is the single most important concept in the course.**

## Function Tools: `@function_tool` Decorator

Tools let agents take actions. The `@function_tool` decorator turns any Python function into a tool.

The SDK automatically:
- Extracts the function name as the tool name
- Uses the docstring as the tool description
- Generates a JSON schema from type hints

In [ ]:
from agents import Agent, Runner, function_tool

@function_tool
def add(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

@function_tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together."""
    return a * b

# Create an agent with tools
calculator = Agent(
    name="Calculator",
    instructions="You are a calculator. Use your tools to perform calculations.",
    model="gpt-4.1",
    tools=[add, multiply]
)

# The agent will use the tools as needed
result = Runner.run_sync(calculator, "What is 7 + 3?")
print(result.final_output)

In [ ]:
# Multi-step calculation (demonstrates the agent loop)
result = Runner.run_sync(calculator, "What is (5 + 3) * 4?")
print(result.final_output)

Notice how the agent:
1. Called `add(5, 3)` → got 8
2. Called `multiply(8, 4)` → got 32
3. Returned the final answer

This is the **agent loop** in action!

## Viewing the Tool Schema

Let's see what the SDK generates automatically from our type hints.

In [ ]:
from agents import FunctionTool
import json

# Inspect the generated schema
for tool in calculator.tools:
    if isinstance(tool, FunctionTool):
        print(f"Tool: {tool.name}")
        print(f"Description: {tool.description}")
        print(f"Schema: {json.dumps(tool.params_json_schema, indent=2)}")
        print()

The SDK automatically created a JSON schema with:
- Parameter names (`a`, `b`)
- Types (`integer`)
- Required fields

This schema tells the LLM how to call the tool correctly.

## Real-World Tools: File Operations

Let's build something more practical - an agent that can read and write files.

In [ ]:
@function_tool
def read_file(file_path: str) -> str:
    """Reads the content of a text file and returns it as a string."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"

@function_tool
def write_file(file_path: str, content: str) -> str:
    """Writes the provided content into a text file."""
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content)
        return "File written successfully."
    except Exception as e:
        return f"Error writing file: {e}"

file_agent = Agent(
    name="FileAgent",
    instructions="You are a helpful assistant that can read from and write to files.",
    model="gpt-4.1",
    tools=[read_file, write_file]
)

In [ ]:
# Write a file
result = Runner.run_sync(
    file_agent, 
    "Write a short haiku about coding to a file called haiku.txt"
)
print(result.final_output)

In [ ]:
# Read it back
result = Runner.run_sync(file_agent, "Read haiku.txt and tell me what it says")
print(result.final_output)

## Pydantic Validation for Complex Tools

For tools with complex parameters, use Pydantic models. This gives you:
- Automatic validation
- Rich type information in the schema
- Field descriptions for the LLM

In [ ]:
from pydantic import BaseModel, Field

class PlotData(BaseModel):
    """Data for creating a simple plot."""
    x_values: list[float] = Field(description="The x-axis values")
    y_values: list[float] = Field(description="The y-axis values")
    title: str = Field(description="The title of the plot")
    color: str = Field(description="The color of the line (e.g., 'blue', 'red')")

@function_tool
def create_plot(data: PlotData) -> str:
    """Create a simple line plot with the given data."""
    # In a real scenario, you'd use matplotlib here
    return f"Created plot '{data.title}' with {len(data.x_values)} points in {data.color}"

In [ ]:
# Let's see the generated schema
print(json.dumps(create_plot.params_json_schema, indent=2))

In [ ]:
plotter = Agent(
    name="Plotter",
    instructions="You create plots based on user requests.",
    model="gpt-4.1",
    tools=[create_plot]
)

result = Runner.run_sync(
    plotter, 
    "Create a plot of the squares from 1 to 5 (1, 4, 9, 16, 25) with a blue line"
)
print(result.final_output)

## Built-in Tools: WebSearchTool

The SDK includes several built-in tools. `WebSearchTool` lets agents search the web.

In [ ]:
from agents.tool import WebSearchTool

researcher = Agent(
    name="Researcher",
    instructions="You are a research assistant. Search the web to answer questions. Be concise.",
    model="gpt-4.1",
    tools=[WebSearchTool()]
)

# Note: This makes a real web search and incurs API costs
result = Runner.run_sync(researcher, "What is the latest Python version?")
print(result.final_output)

## Async Execution

For production use, prefer async execution with `Runner.run()`.

In [ ]:
import asyncio

async def main():
    result = await Runner.run(calculator, "What is 100 + 23?")
    print(result.final_output)

asyncio.run(main())

## Key Takeaways

1. **Agent = LLM + Instructions + Tools** running in a loop
2. **The agent loop**: tool call → execute → feed back → repeat until done
3. **`@function_tool`** decorator automatically generates JSON schemas from type hints
4. **Pydantic models** provide rich validation and descriptions for complex tools
5. **`Runner.run_sync()`** for simple scripts, `Runner.run()` for async production code

Next up: **Structured Output** - making agents return typed, validated responses.